Difficulties we may encounter based on initial tests:
1. How should we retrieve job listings with RAG, and how should the results be included in the conversation context?
2. How to manage user uploading resume? Should they copy it directly into the conversation, or should it be uploaded through the GUI and included
with the LLM when necessary
3. It seems like sometimes the LLM is not great at doing the skills gap analysis on its own

In [1]:
from openai import OpenAI
import yaml
from pathlib import Path

In [2]:
SYSTEM_PROMPT = "You are a helpful job finder and resume analyzer."

In [3]:
from openai import BadRequestError


def eval_agent(client: OpenAI) -> float:
    """
    Evaluate the agent on the given benchmark YAML file.
    """
    for p in Path(".").glob("*.yaml"):
        print(p)
        print()

        conversation = [{'role': 'system', 'content': SYSTEM_PROMPT}]
        with open(p, "r") as file:
            steps = yaml.safe_load(file)
        try:
            for n, step in enumerate(steps):
                conversation.append({'role': 'user', 'content': step['prompt']})
                response = client.chat.completions.create(
                    messages = conversation,
                    model = "meta-llama/Meta-Llama-3.1-8B-Instruct",
                    temperature=0.2,
                ).choices[0].message.content
                conversation.append({'role': 'assistant', 'content': response})
                print(f'Q: {step["prompt"]}\n')
                print(f'A: {response}\n')
                print()
        except BadRequestError as e:
            print('Context limit reached.')
            continue

        print()
        print('-------------------------------------')
        print()
            

In [4]:
BASE_URL = "http://199.94.61.113:8000/v1/"
API_KEY = "tarallo.n@northeastern.edu:OQEgetkQ6LBofgWW4jsC"
client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

In [5]:
eval_agent(client)

test1.yaml

Q: John Doe
Email: john.doe@example.com Phone: (555) 123-4567 LinkedIn: linkedin.com/in/johndoeGitHub: github.com/johndoe

Professional Summary
Innovative software engineer with 5+ years of experience in full stack development, specializing in designing and deploying scalable web applications.

Skills
Languages: JavaScript, Python, Java
Frameworks: React, Node.js, Django
Databases: MySQL, MongoDB
Tools: Git, Docker
Cloud: AWS, Azure

Experience
Senior Software Engineer Tech Solutions Inc. | Boston, MA Jan 2022 - Present
- Developed e-commerce platform with React and Node.js, boosting user engagement by 30%.
- Implemented CI/CD with Jenkins, cutting deployment time by 40%.

Software Engineer Innovative Apps LLC | Boston, MA Jun 2018 - Dec 2021
- Maintained RESTful APIs using Django, improving system efficiency by 25%.
- Collaborated with cross-functional teams on software solutions.

Education
B.S. in Computer Science Northeastern University | Boston, MA Graduated: May 2018
